In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import pickle
import os

from matplotlib import pyplot as plt
from random import shuffle
path="csv_files/"

In [ ]:
from utils.dicts_transformer import *

## Functions

In [ ]:
def transform_coordinates(x1,y1,x2,y2,dimX,dimY):
    centerX=(x1+x2)/2
    centerY=(y1+y2)/2
    width=x2-x1
    height=y2-y1
    #to relative values:
    centerX=centerX/dimX
    centerY=centerY/dimY
    width=width/dimX
    height=height/dimY
    
    return centerX,centerY,width,height
    

In [ ]:
def zerofix(string):
    s=string.split("/")[-1].split("e")[-1]
    rest=string.split("/")[:-1]
    result="/".join(rest)
    i=int(s)-1
    st=result+'/frame'+str(i).zfill(4)
    return st

In [ ]:
def balanced(labelcounter, files_per_class):
    m=np.min(labelcounter)
    filelist=[]
    for j,files_class in enumerate(files_per_class):
        shuffle(files_class)
        for i,s in enumerate(files_class):
            if m-1<=i:
                break;
            if s not in filelist:    
                filelist.append(s)
        print(j,i)
    return filelist

# Creating labelfiles

In [ ]:
datafiles=[]
basepath='/data/Images/'
#datafiles.append(basepath+'test_environment')
datafiles.append(basepath+'signs001_manually_labeled')
datafiles.append(basepath+'signs003_manually_labeled')
datafiles.append(basepath+'signs004_manually_labeled')
datafiles.append(basepath+'signs006_bosch_manually_labeled')
datafiles.append(basepath+'2019-01-09-13-00-05')
datafiles.append(basepath+'2019-01-09-13-01-41')
datafiles.append(basepath+'2019-01-09-13-05-43')
datafiles.append(basepath+'2019-01-09-13-24-37')
datafiles.append(basepath+'2019-01-09-13-55-01')
datafiles.append(basepath+'2019-01-20-15-59-41')

# Zone fillers
datafiles.append(basepath+'VID_20190125_181012')
datafiles.append(basepath+'VID_20190125_181207')
datafiles.append(basepath+'VID_20190125_181313')
datafiles.append(basepath+'VID_20190125_181630')
datafiles.append(basepath+'VID_20190125_181902')

## Blur
datafiles.append(basepath+'signs001_manually_labeled_blur')
datafiles.append(basepath+'signs003_manually_labeled_blur')
datafiles.append(basepath+'signs004_manually_labeled_blur')
datafiles.append(basepath+'signs006_bosch_manually_labeled_blur')
datafiles.append(basepath+'2019-01-09-13-00-05_blur')
datafiles.append(basepath+'2019-01-09-13-01-41_blur')
datafiles.append(basepath+'2019-01-09-13-05-43_blur')
datafiles.append(basepath+'2019-01-09-13-24-37_blur')
datafiles.append(basepath+'2019-01-09-13-55-01_blur')
datafiles.append(basepath+'2019-01-20-15-59-41_blur')
datafiles.append(basepath+'VID_20190125_181012_blur')
datafiles.append(basepath+'VID_20190125_181207_blur')
datafiles.append(basepath+'VID_20190125_181313_blur')
datafiles.append(basepath+'VID_20190125_181630_blur')
datafiles.append(basepath+'VID_20190125_181902_blur')

#datafiles.append(basepath+'bosch_2018_16_36') Bug in labels
datafiles.append(basepath+'bosch_2018_17_08')
datafiles.append(basepath+'bosch_2018_17_24')
datafiles.append(basepath+'bosch_2018_17_54')

#
datafiles.append(basepath+'Datasets/GTSDB')
#datafiles.append(basepath+'Datasets/STS')
#datafiles.append(basepath+'Datasets/BTSDB')
#datafiles.append(basepath+'Datasets/rtsd-public/detection/rtsd-d1-frames')
#datafiles.append(basepath+'Datasets/rtsd-public/detection/rtsd-d2-frames')
#datafiles.append(basepath+'Datasets/rtsd-public/detection/rtsd-d3-frames')


#zerobase=['bosch_2018_16_36','bosch_2018_17_54','bosch_2018_17_24','bosch_2018_17_08','2019-01-09-13-00-05','2019-01-09-13-01-41','2019-01-09-13-05-43','2019-01-09-13-24-37','2019-01-09-13-55-01']
zerobase=[]
jpglist=['VID_20190125_181902','VID_20190125_181630','VID_20190125_181313','VID_20190125_181207','VID_20190125_181012','Datasets/rtsd-public/detection/rtsd-d3-frames','Datasets/rtsd-public/detection/rtsd-d2-frames','Datasets/rtsd-public/detection/rtsd-d1-frames','bosch_2018_17_54','bosch_2018_17_24','bosch_2018_17_08','2019-01-09-13-00-05','2019-01-09-13-01-41','2019-01-09-13-05-43','2019-01-09-13-24-37','2019-01-09-13-55-01','2019-01-20-15-59-41', 'Datasets/STS']
ppmlist=['Datasets/GTSDB']
jp2list=['Datasets/BTSDB']

In [ ]:
files=[]
oldFile=None
num_Zero=2000
counter=0
frames = []
zero_class=sign_name_carolo_trans.get(43)
labelcounter=np.zeros((31,))
files_per_class=np.empty((31,),dtype=object)
for i,v in enumerate(files_per_class): files_per_class[i]=[]
    
#iterate over all folder from datasets
for folder in datafiles:
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            print(filename)
            df = pd.read_csv(folder+'/'+filename)
            df['ClassId'].replace(sign_name_carolo_trans, inplace=True)
            for index, row in df.iterrows():           
                classid=int(row['ClassId'])
                centerX,centerY,width,height=transform_coordinates(row['Roi.X1'],row['Roi.Y1'],row['Roi.X2'],row['Roi.Y2'],row['Width'],row['Height'])
                #file=row['Filename'].split('.')[0]
                file=''.join(row['Filename'].split('.')[0:-1])
                if any(st in folder for st in zerobase):
                    file=zerofix(file)
                    file=file.split('/')[-1]
                labelcounter[classid]+=1
                if classid!=zero_class:
                    if file==oldFile:
                        #print(centerX,centerY,width,height)
                        f = open(folder+'/'+file+'.txt',"a") 
                        f.write(str(classid)+' '+str(centerX)+' '+str(centerY)+' '+str(width)+' '+str(height)+'\n')
                        f.close()
                        files_per_class[classid].append(folder+'/'+file)
                    else:
                        f = open(folder+'/'+file+'.txt',"w") 
                        f.write(str(classid)+' '+str(centerX)+' '+str(centerY)+' '+str(width)+' '+str(height)+'\n')
                        f.close()
                        #print(folder+'/'+file)
                        files.append(folder+'/'+file)
                        files_per_class[classid].append(folder+'/'+file)
                elif classid==20 and counter<num_Zero:
                    f = open(folder+'/'+file+'.txt',"w") 
                    f.close()
                    counter+=1
                    files.append(folder+'/'+file)
                    files_per_class[classid].append(folder+'/'+file)
                oldFile=file
                
                #frames_added_april.append(pd.read_csv(filename))

In [ ]:
for i in range(len(sign_name_carolo_dict_new)):
    print(str(labelcounter[i])+'  '+sign_name_carolo_dict_new[i])

In [ ]:
test=np.empty((31,),dtype=object)
for i,v in enumerate(test): test[i]=[]

In [ ]:
for i,e in enumerate(files_per_class[0]):
    print(i)

In [ ]:
balance=False
if balance==True:
    files=balanced(labelcounter,files_per_class)
    print("Minimum Labels is:")
    print(np.min(labelcounter))
    print("Dataset is balanced to that number")

# Train & Test files

In [ ]:
#X_train,X_test=train_test_split(files, shuffle=False)

num_of_rows = round((len(files)) * 0.8)
files_np=np.array(files)
np.random.shuffle(files_np) 
X_train= files_np[:num_of_rows] 
X_test = files_np[num_of_rows:] 

file = open(basepath+'train.txt',"w")
for line in X_train:
    #if any(st in line for st in zerobase):
     #   line=zerofix(line)
    if any(st in line for st in jpglist):
        file.write(str(line)+'.jpg\n')
    elif any(st in line for st in ppmlist):
        file.write(str(line)+'.ppm\n')
    elif any(st in line for st in jp2list):
        file.write(str(line)+'.jp2\n')
    else:
        file.write(str(line)+'.png\n')
file.close()
    
file = open(basepath+'test.txt',"w")
for line in X_test:
    #if any(st in line for st in zerobase):
    #    line=zerofix(line)
    if any(st in line for st in jpglist):
        file.write(str(line)+'.jpg\n')
    elif any(st in line for st in ppmlist):
        file.write(str(line)+'.ppm\n')
    elif any(st in line for st in jp2list):
        file.write(str(line)+'.jp2\n')
    else:
        file.write(str(line)+'.png\n')
file.close()
    

In [ ]:
file = open(basepath+'signs.names',"w")
for num in range(len(sign_name_carolo_trans)-1):
    file.write(str(sign_name_carolo_dict_new.get(num)))
file.close()

In [ ]:
with open('../sign_recognition/dict/sign_names_dict.pkl', 'rb') as f:
    sign_names = pickle.load(f)

In [ ]:
sign_names[0]

In [ ]:
len(sign_name_carolo_trans)